In [1]:
import numpy as np
import matplotlib

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from scipy.integrate import solve_ivp
from scipy.optimize import root_scalar

In [2]:
def f(a,b,r):
    return a * (1 + b*r) * np.exp(-b*r)

In [3]:
def g(a,b,r):
    return (1 + a * (1 + b*r) * np.exp(-b*r))**3 / (1 + a * (1 + b*r + (b*r)**2) * np.exp(-b*r))

In [4]:
def T(a,b,r):
    return 2*np.pi * np.sqrt(np.power(r,3) / (1+f(a,b,r)))

In [5]:
def solve_radius(tmin, tmax, r0, a, b):
    return solve_ivp(lambda t,y: - g(a,b,y)/y**3, [tmin,tmax], [r0], dense_output=True)

In [6]:
def invert_period(a,b,p):
    guess = np.power(p/(2*np.pi),2/3)
    sol = root_scalar(lambda x: T(a,b,x) - p, bracket=[guess*0.5, guess*1.5])
    return sol.root

In [7]:
def period_plot(a,b):
    p0 = 30
    r0 = invert_period(a,b,p0)
    tmin, tmax = 0, 14

    sol = solve_radius(tmin,tmax,r0,a,b)
    t = np.linspace(tmin,tmax,100000)

    r1 = sol.sol(t)[0]
    r2 = np.power((r0**4 - 4*t),1/4)

    p0 = T(a,b,r0)
    p1 = np.vectorize(T)(a,b,r1) / p0
    p2 = 2*np.pi * np.power(((p0/(2*np.pi))**(8/3) - 4*t),3/8) / p0
    
    return t, p1, p2

def plot(params):
    styles = ["--","-.",":","--"]
    
    plots = [period_plot(a,b) for (a,b) in params]
    
    golden = (5**.5-1)/2
    width_pt = 448
    scale = 0.9
    dpi = 72.7
    width = scale*width_pt / dpi
    height = golden * width
    
    fig, ax = plt.subplots(1,figsize=(width,height))

    ax.plot(plots[0][0], plots[0][2], "-", label="metric")

    for i in range(len(params)):
        ax.plot(plots[i][0], plots[i][1], styles[i], label=f"area metric [$\\beta = {params[i][0]}$, $\\mu = {params[i][1]}$]")
 
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.1))
        
    ax.set_xlabel("coordinate time $t$")
    ax.set_ylabel(r"orbital period $P / P_0$")
        
    ax.legend(loc="lower left")
    
    return fig, ax

fig1, ax1 = plot([(0.2,1), (1,1), (4,1)])
ax1.set_title("Spin-up of the binary star for different constants $\\beta$")
fig1.savefig("plot1.pgf")

fig2, ax2 = plot([(1,1), (1,2), (1,3)])
ax2.set_title("Spin-up of the binary star for different constants $\\mu$")
fig2.savefig("plot2.pgf")